# Plotting Excel in Python

Cause sometimes you need to work with others more comfortable with Microsoft Office tools

## Resources for help
[openpyxl library](https://openpyxl.readthedocs.io/en/stable/)

 - [OpenPyXL charts documentation](https://openpyxl.readthedocs.io/en/stable/charts/bar.html)

[Automate the Boring Stuff with Python Ch. 12](https://automatetheboringstuff.com/chapter12/)

Youtube vid:
https://www.youtube.com/watch?v=8z61LhMsyDM

## Make some fake data 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl.chart import Reference
from openpyxl.chart.data_source import NumDataSource
from openpyxl.chart.data_source import NumRef
from openpyxl.chart.error_bar import ErrorBars
from openpyxl.chart.series_factory import SeriesFactory
from openpyxl.chart import BarChart

import utils

In [2]:
ngroups = 6

means = [utils.random_heights() for i in range(ngroups)]
stds =  [utils.random_heights()/10 for i in range(ngroups)]
names = utils.random_names(means[0])

In [3]:
z = dict()
dicts = [{f'group {i} mean': mean, f'group {i} std': std} for mean, std, i in zip(means, stds, range(len(means)))]
[z.update(d) for d in dicts]
df = pd.DataFrame(z, index = names)
df

,group 0 mean,group 0 std,group 1 mean,group 1 std,group 2 mean,group 2 std,group 3 mean,group 3 std,group 4 mean,group 4 std,group 5 mean,group 5 std
bly,0.382261,0.056637,0.132090,0.102985,0.560602,0.040105,0.619217,0.059136,0.509123,0.035278,1.513015,0.043684
ytc,2.442051,0.109679,0.852712,0.036745,0.372749,0.053047,0.336591,0.007514,0.602521,0.062069,0.517119,0.038464
yaa,2.591870,0.214162,0.326223,0.165174,0.322672,0.129101,1.542011,0.023878,1.497023,0.109981,0.124170,0.007184
izq,1.620975,0.051761,0.190076,0.074457,1.037734,0.208695,1.085066,0.099313,0.939723,0.140178,0.758618,0.002476
nxn,0.529553,0.139753,0.416810,0.105088,0.584182,0.064066,1.137892,0.049405,1.177015,0.040173,1.013285,0.028430
dfs,1.717542,0.054463,0.073150,0.169340,0.019679,0.066583,0.327596,0.019088,0.283259,0.063793,1.369350,0.077416
yae,0.158410,0.240478,1.448997,0.023741,0.554473,0.029241,0.810482,0.104377,0.793596,0.137458,0.012680,0.048766
apo,0.352073,0.160417,0.728218,0.034424,1.292122,0.032463,0.639092,0.184529,0.669954,0.082093,0.789839,0.065426


In [4]:
cols = df.columns.tolist()
cols

['group 0 mean',
 'group 0 std',
 'group 1 mean',
 'group 1 std',
 'group 2 mean',
 'group 2 std',
 'group 3 mean',
 'group 3 std',
 'group 4 mean',
 'group 4 std',
 'group 5 mean',
 'group 5 std']

In [5]:
rearrange_cols = [*cols[0::2], *cols[1::2]]
rearrange_cols

['group 0 mean',
 'group 1 mean',
 'group 2 mean',
 'group 3 mean',
 'group 4 mean',
 'group 5 mean',
 'group 0 std',
 'group 1 std',
 'group 2 std',
 'group 3 std',
 'group 4 std',
 'group 5 std']

In [6]:
rearranged_df = df[rearrange_cols]

In [7]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(rearranged_df, index=True, header=True):
    ws.append(r)

In [8]:
name_ref = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=len(means[0])+2)
col_idx = 2
mean_refs = [Reference(ws, min_col=i, min_row=3, max_col=i, max_row=len(means[0])+2) for i in range(col_idx, col_idx+ngroups)]
col_idx+=ngroups
std_refs = [Reference(ws, min_col=i, min_row=3, max_col=i, max_row=len(means[0])+2) for i in range(col_idx, col_idx+ngroups)]

In [9]:
means_series = [SeriesFactory(mean_ref, title=f'group {i}') for i, mean_ref in enumerate(mean_refs)]

In [10]:
for i, std_ref in enumerate(std_refs):
    eBarsNumDataSource = NumDataSource(NumRef(std_ref))
    means_series[i].errBars = ErrorBars(errDir='y', errValType='cust', plus=eBarsNumDataSource, minus=eBarsNumDataSource)

In [11]:
chartObj = BarChart()
[chartObj.append(means) for means in means_series]
chartObj.title = 'Cytokine array results'
chartObj.set_categories(name_ref)
chartObj.y_axis.title = "Relative Intensity"
chartObj.x_axis.title = "Cytokine"

In [12]:
ws.add_chart(chartObj, 'B12')

In [13]:
wb.save('sampleChart.xlsx')